# Map colouring
Using the GraphILP API to colour the map of all districts in Germany with as few colours as possible such that adjacent districts get different colours.

In [ ]:
import geopandas as gpd
from shapely.geometry import LineString

In [ ]:
import networkx as nx

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from matplotlib import pyplot as plt

## Get the data: a map of districts in Germany

In [ ]:
data = gpd.read_file("/mnt/data/gis/vg250_0101.gk3.shape.ebenen/vg250_ebenen/VG250_KRS.shp")

Let's draw a map of the districts.
Your job is to colour each district in such a way that
* you use as few colours as possible,
* adjacent district are coloured with different colours.

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
plt.axis('off')
data.plot(ax=ax);

## Set up a graph
We will cast this as a problem on graphs by creating a graph in which
* the vertices correspond to the districts and
* there is an edge between two vertices if the corresponding districts are adjacent

In [ ]:
# to help visualise the graph, we compute the centroid of each district
centroids = data['geometry'].apply(lambda x: x.centroid)

In [ ]:
# we use pairwise intersection of districts to find out whether they are adjacent
intersect = gpd.sjoin(data, data)

In [ ]:
# as a result, we can extract the edges to be used in our graph
links = [(row[0], row[1].index_right) for row in intersect.iterrows() if row[0] != row[1].index_right]

In [ ]:
# let us create some geometry from the edges, so that we can plot them
lines = [LineString((centroids.loc[a], centroids.loc[b])) for (a, b) in links]
lines_df = gpd.GeoDataFrame(geometry=lines)

In [ ]:
# now we can visualise the graph on top of our map
fig, ax = plt.subplots(figsize=(12,8))
plt.axis('off')
data.plot(ax=ax)
centroids.plot(color='pink', ax=ax)
lines_df.plot(color='red', ax=ax);

In [ ]:
# create a graph from our data
mygraph = nx.Graph()
mygraph.add_edges_from(links)

## Set up and solve the problem using GraphILP API

In [ ]:
import sys
sys.path.append("../..")
from graphilp.imports import networkx as imp_nx
from graphilp.partitioning import min_vertex_coloring
from graphilp.sub_super import max_clique_cover as max_clique

In [ ]:
G = imp_nx.read(mygraph)

In [ ]:
model = min_vertex_coloring.createModel(G)

In [ ]:
model.optimize()

In [ ]:
color_assignment, node_to_col = min_vertex_coloring.extractSolution(G, model)

## Visualise the solution

In [ ]:
data['colors'] = data.apply(lambda row: node_to_col[row.name], axis=1)

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
plt.axis('off')
data.plot(column='colors', cmap='Set1', ax=ax);

## Analyse the solution

In [ ]:
# check how many colours we need
set(node_to_col.values())

Wait! Shouldn't four colours suffice for every planar map according to the famous <a href="https://en.wikipedia.org/wiki/Four_color_theorem">Four colour theorem</a>?

In [ ]:
# perhaps our graph is not planar?
nx.algorithms.planarity.check_planarity(mygraph)

Indeed, now let us try to find a region that is to blame for this!
We will do so by invoking another function of our library to find a <a href="https://en.wikipedia.org/wiki/Clique_problem">maximum size clique</a>.

In [ ]:
model = max_clique.createModel(G)

In [ ]:
model.optimize()

In [ ]:
clique = max_clique.extractSolution(G, model)

In [ ]:
data.iloc[clique]

In [ ]:
# here are all the existing links between the nodes in our clique
clique_links = [(a, b) for (a, b) in links if a in clique and b in clique]

In [ ]:
# let us create some geometry from these edges, too, so that we can plot them
clique_lines = [LineString((centroids.loc[a], centroids.loc[b])) for (a, b) in clique_links]
clique_lines_df = gpd.GeoDataFrame(geometry=clique_lines)

In [ ]:
# now we can visualise the clique
fig, ax = plt.subplots(figsize=(12,8))
plt.axis('off')
data.iloc[clique].plot(column='colors', cmap='Set1', ax=ax)
centroids.iloc[clique].plot(color='pink', ax=ax)
clique_lines_df.plot(color='red', ax=ax);